In [ ]:
from langchain.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.llms import CTransformers, Ollama

: 

In [ ]:
##from dotenv import load_dotenv
import os
##load_dotenv()

: 

In [ ]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

: 

In [ ]:
#DATA INGESTION
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

: 

In [ ]:
pip install pypdf

: 

In [ ]:
extracted_data = load_pdf(data=r"data")

: 

In [ ]:
len(extracted_data)

: 

In [ ]:
#DATA TRANSFORMATION
#CREATING TEXT CHUNKS
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

: 

In [ ]:
docs = text_split(extracted_data=extracted_data)

: 

In [ ]:
len(docs)

: 

In [ ]:
#Use embedding model
def hugging_face_embed_docs():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

def ollama_embed_docs():
    embeddings = OllamaEmbeddings(model="llama2")
    return embeddings

: 

In [ ]:
pip install sentence-transformers

: 

In [ ]:
embeddings = ollama_embed_docs()
hugging_embeddings = hugging_face_embed_docs()

: 

In [ ]:
embeddings

: 

In [ ]:
hugging_embeddings

: 

In [ ]:
pip install faiss-cpu



: 

In [ ]:
from langchain.vectorstores import FAISS

: 

In [ ]:
vector_store = FAISS.from_documents(docs, hugging_embeddings)


: 

In [ ]:
vector_store

: 

In [ ]:
sample_ans = vector_store.similarity_search("What is an allergy", k=3)
sample_ans

: 

In [ ]:
prompt_template = """
    Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    If you know the answer, answer it in a crisp and descriptive way. Use around 5 to 6 sentences to explain.
    Context: {context}
    Question: {question}
    Only return the helpful answer below and nothing else.
    At the end of each answer which you know, don't end it with any statement thanking the user for asking you the question. Just end your answer normally without thanking the user. Just anser what is necessary. No thank you to the user is needed
    If the user tells "Thank you", or anything related to it, then reply in short, thanking the user to ask questions to you and telling him to consult you again in the future in case of any help.
    Assistant : 
    """

: 

In [ ]:
PROMPT = PromptTemplate(template = prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt" : PROMPT}

: 

In [ ]:
# llm = CTransformers(model = r"C:\Users\Arjo\.ollama\models\blobs\sha256-8934d96d3f08982e95922b2b7a2c626a1fe873d7c3b06e8e56d7bc0a1fef9246", model_type='llama2', config={'max_new_tokens' : 512, 'temperature' : 0.8})

: 

In [ ]:
llm = Ollama(model="phi3:mini", temperature=0.8, num_predict=1024)

: 

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True,
    retriever = vector_store.as_retriever(search_kwargs = {'k' : 2})
)

: 

In [ ]:
while True:
    user_input = input(f"Input Prompt : ")
    result = qa({'query' : user_input})
    print("Response : ", result['result'])

: 

: 